In [187]:
%load_ext autoreload
%autoreload 2
%ls data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
binary_tweets.csv  multiclass_tweets.csv  README.md    tweets_normalized.txt
first_scrape.json  parsed_start_0.json    tweets.json


In [206]:
import pandas as pd
from util import const, reader

In [189]:
# unused
from collections import namedtuple
def to_namedtuple(dict_): 
  return namedtuple(dict_.__name__, dict_.keys())

In [190]:
df_original = pd.read_json(const.Path.TWEETS_CRAWLED_JSON)
df = pd.read_json(const.Path.TWEETS_SN_PARSED_JSON)

print("Non-null:", len(df.dropna(subset=['output'])))
print("Equal:", (df['output'].isnull() == \
                 df['output'].isnull() & df['input'].isnull()).all())
df = df.dropna()
print("Total:", len(df))

Non-null: 116882
Equal: True
Total: 116882


In [191]:
# expand each dictionary to columns in a DF 
def expand_to_columns(df, key):
  key = [key] if isinstance(key, str) else key
  expanded = [df[k].apply(pd.Series) for k in key]
  for k, e in zip(key, expanded):
    e.columns = ['{}_{}'.format(k, c) for c in e.columns]
  return pd.concat([df.drop(key, axis=1)] + expanded, axis=1)

In [192]:
%time df_expanded = expand_to_columns(df_original, ['user', 'body'])

CPU times: user 50.8 s, sys: 577 ms, total: 51.3 s
Wall time: 51.7 s


In [193]:
df_joined = df.join(df_expanded, on='index')

In [198]:
df_joined[df_joined.duplicated(['body_content'], keep=False)]\
  .sort_values(['body_content'])

,index,input,output,children,created_at,favorite_count,id,label,media,parents,reply_count,retweet_count,user_handle,user_id,user_is_verified,user_name,body_atreplies,body_content,body_hashtags,body_links
77265,77265,@user @user @user @user @user @user ...,"[{'category': '', 'pos_tag': 'RB', 'head': 2, ...",NaN,2016-03-21 12:46:00,1.0,7.120028e+17,0.0,NaN,NaN,0.0,1.0,ZeroCheff,169010330,False,Zeren Somunkıran,"[@EU_Commission, @Reuters, @cnni, @BBCWorld, @...",\n#StopChildRapeInTurkey \n@EU_Commission \n@R...,[#StopChildRapeInTurkey],NaN
75953,75953,students' unions banning things - as covered b...,"[{'category': '', 'pos_tag': 'NNS', 'head': 2,...",NaN,2016-03-21 12:46:00,1.0,7.120024e+17,0.0,NaN,NaN,0.0,1.0,ZeroCheff,356781591,False,Zeren Somunkıran,"[@EU_Commission, @Reuters, @cnni, @BBCWorld, @...",\n#StopChildRapeInTurkey \n@EU_Commission \n@R...,[#StopChildRapeInTurkey],NaN
45781,45781,@user @user send more children to prison i say,"[{'category': '', 'pos_tag': 'RB', 'head': 1, ...",NaN,2016-03-15 05:33:00,0.0,7.097191e+17,1.0,NaN,NaN,0.0,0.0,ThomasLiesener,130791667,False,Thomas Liesener,[@engadget],"Next for iRobot, a cute $199 kitchen and bath...",NaN,[engadget.com/2016/03/15/iro]
50176,50176,"spoiler alert: it was when us had more ""social...","[{'category': '', 'pos_tag': 'NN', 'head': 1, ...",NaN,2016-03-15 05:33:00,0.0,7.097191e+17,1.0,NaN,NaN,0.0,0.0,ThomasLiesener,130791667,False,Thomas Liesener,[@engadget],"Next for iRobot, a cute $199 kitchen and bath...",NaN,[engadget.com/2016/03/15/iro]
102011,102011,@hashtag 737 heading to russia’s @hashtag ...,"[{'category': '', 'pos_tag': 'NFP', 'head': 3,...",NaN,2016-03-21 16:52:00,16.0,7.120645e+17,0.0,NaN,NaN,0.0,6.0,jaavedjaaferi,90628174,True,Jaaved Jaaferi,[@WorldTruthTV],RT @WorldTruthTV: CDC Warns: Hair Bands From ...,NaN,[fb.me/2l2keL6bf]
101939,101939,details: all 61 people on board @hashtag kil...,"[{'category': '', 'pos_tag': 'NNS', 'head': -1...",NaN,2016-03-21 16:52:00,16.0,7.121450e+17,0.0,NaN,NaN,0.0,6.0,jaavedjaaferi,4071345918,True,Jaaved Jaaferi,[@WorldTruthTV],RT @WorldTruthTV: CDC Warns: Hair Bands From ...,NaN,[fb.me/2l2keL6bf]
64128,64128,video: trump emotional moment with former mis...,"[{'category': '', 'pos_tag': 'NN', 'head': -1,...","[{'date': '2016-03-27 05:00:00', 'user_id': '3...",2016-03-27 04:40:00,1.0,7.140574e+17,0.0,NaN,NaN,2.0,1.0,crankyallAnimal,3160381099,False,dameCranky,"[@GatesofVienna, @havenaar64]","""@GatesofVienna: Gates of Vienna: German Media...",NaN,[gatesofvienna.net/2016/03/german]
64141,64141,rt: @user :britain's royal navy to hold 'rob...,"[{'category': '', 'pos_tag': 'NN', 'head': -1,...","[{'date': '2016-03-27 05:00:00', 'user_id': '3...",2016-03-27 04:40:00,1.0,7.140545e+17,0.0,NaN,NaN,2.0,1.0,crankyallAnimal,631742193,False,dameCranky,"[@GatesofVienna, @havenaar64]","""@GatesofVienna: Gates of Vienna: German Media...",NaN,[gatesofvienna.net/2016/03/german]
74971,74971,. @user saudi attacks on yemen have claimed t...,"[{'category': '', 'pos_tag': 'NNP', 'head': 3,...",NaN,2016-03-15 23:19:00,0.0,7.099874e+17,0.0,NaN,NaN,0.0,1.0,Isaackcheboiwo,1903978172,False,Isaack,[@RT_com],"""@RT_com: 'Treasure hiker': Extremely rare 200...",[#Israel],[on.rt.com/76zo]
81530,81530,"@user if western logic apply, britain are res...","[{'category': '', 'pos_tag': 'RB', 'head': 8, ...",NaN,2016-03-15 23:19:00,0.0,7.099874e+17,0.0,NaN,NaN,0.0,1.0,Isaackcheboiwo,3499405755,False,Isaack,[@RT_com],"""@RT_com: 'Treasure hiker': Extremely rare 200...",[#Israel],[on.rt.com/76zo]


In [203]:
df_unique = df_joined.drop_duplicates(['input'])
print("Unique tweets:", len(df_unique))

Unique tweets: 98375
